In [1]:
import sys
sys.path.append('/Users/harsh/Work/habitat-sim/')
sys.path.append('/Users/harsh/Work/habitat-api/')

In [2]:
cd ..

/coc/pskynet3/hagrawal9/project/sokoban/habitat-api


In [3]:
import gzip
import habitat
import habitat_sim
import habitat_sim.bindings as hsim
import json
import matplotlib.pyplot as plt
import magnum as mn
import numpy as np
import os
import tqdm 

import habitat_sim

from PIL import Image
from typing import Any, Dict, List, Optional, Union

from habitat import Config, logger
from habitat import make_dataset
from habitat.core.simulator import AgentState
# from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower
from habitat.sims.habitat_simulator.actions import HabitatSimActions
from habitat.sims.habitat_simulator.habitat_simulator import HabitatSim
from habitat.utils.visualizations.utils import observations_to_image, images_to_video
from habitat.utils.geometry_utils import (
    angle_between_quaternions,
    quaternion_from_two_vectors,
)
from habitat.tasks.utils import (
    cartesian_to_polar,
    quaternion_from_coeff,
    quaternion_rotate_vector
)

from habitat_baselines.common.base_trainer import BaseRLTrainer
from habitat_baselines.common.baseline_registry import baseline_registry
from habitat_baselines.common.env_utils import construct_envs
from habitat_baselines.common.environments import get_env_class, SokobanRLEnv
from habitat_baselines.common.rollout_storage import RolloutStorage
from habitat_baselines.common.tensorboard_utils import TensorboardWriter
from habitat_baselines.common.utils import (
    batch_obs,
    generate_video,
    linear_decay,
)
from habitat_baselines.config.default import get_config
from habitat_baselines.rl.ppo import PPO, PointNavBaselinePolicy

from habitat_sim.physics import MotionType
from habitat_sim.attributes import PhysicsObjectAttributes
from habitat_sim.helper import *


EPSILON = 1e-6


def action_to_one_hot(action: int) -> np.array:
    one_hot = np.zeros(len(HabitatSimActions), dtype=np.float32)
    one_hot[action] = 1
    return one_hot

%matplotlib inline

In [4]:
content_path = "data/datasets/sokoban/coda/v1/train/content/val_object_5_scene_10_coda.json.gz"

In [5]:
config = get_config("habitat_baselines/config/sokoban/pickup_order_ddppo_obj_5_scene_100.yaml")

In [6]:
config.defrost()
config.SENSORS = ['RGB_SENSOR', 'DEPTH_SENSOR']
config.NUM_PROCESSES = 1
config.TASK_CONFIG.TASK.SENSORS = ['GRIPPED_OBJECT_SENSOR',
 'CLOSEST_OBJECT_SENSOR']
config.TASK_CONFIG.ENVIRONMENT.MAX_EPISODE_STEPS = 1000
config.TASK_CONFIG.TASK.POSSIBLE_ACTIONS = ['STOP', 'MOVE_FORWARD', 'TURN_LEFT', 'TURN_RIGHT', 'GRAB_RELEASE']
config.TASK_CONFIG.DATASET.CONTENT_SCENES = ['val_object_5_scene_10_coda']
config.TASK_CONFIG.ENVIRONMENT.ITERATOR_OPTIONS.SHUFFLE = False
config.freeze()

In [7]:
dataset = make_dataset(config.TASK_CONFIG.DATASET.TYPE, config=config.TASK_CONFIG.DATASET)

2020-06-30 18:47:44,153 Initializing dataset Sokoban-v0


In [8]:
class ShortestPathFollower:
    r"""Utility class for extracting the action on the shortest path to the
        goal.
    Args:
        sim: HabitatSim instance.
        goal_radius: Distance between the agent and the goal for it to be
            considered successful.
        return_one_hot: If true, returns a one-hot encoding of the action
            (useful for training ML agents). If false, returns the
            SimulatorAction.
    """

    def __init__(
        self, sim: HabitatSim, goal_radius: float, return_one_hot: bool = True
    ):
        assert (
            getattr(sim, "geodesic_distance", None) is not None
        ), "{} must have a method called geodesic_distance".format(
            type(sim).__name__
        )

        self._sim = sim
        self._max_delta = self._sim.config.FORWARD_STEP_SIZE - EPSILON
        self._goal_radius = goal_radius
        self._step_size = self._sim.config.FORWARD_STEP_SIZE

        self._mode = (
            "exact_gradient"
            if getattr(sim, "get_straight_shortest_path_points", None)
            is not None
            else "approximate_gradient"
        )
        
        self._mode = "approximate_gradient"
        
        self._return_one_hot = return_one_hot

    def _get_return_value(self, action) -> Union[int, np.array]:
        if self._return_one_hot:
            return action_to_one_hot(action)
        else:
            return action

    def get_next_action(
        self, goal_pos: np.array
    ) -> Optional[Union[int, np.array]]:
        """Returns the next action along the shortest path.
        """
        if (
            self._sim.geodesic_distance(
                self._sim.get_agent_state().position, [goal_pos]
            )
            <= self._goal_radius
        ):
            return None

        max_grad_dir = self._est_max_grad_dir(goal_pos)
        
        if max_grad_dir is None:
            return self._get_return_value(HabitatSimActions.MOVE_FORWARD)
            # return None
        
        return self._step_along_grad(max_grad_dir)

    def _step_along_grad(
        self, grad_dir: np.quaternion
    ) -> Union[int, np.array]:
        current_state = self._sim.get_agent_state()

        alpha = angle_between_quaternions(grad_dir, current_state.rotation)
        if alpha <= np.deg2rad(self._sim.config.TURN_ANGLE) + EPSILON:
            return self._get_return_value(HabitatSimActions.MOVE_FORWARD)
        else:
            sim_action = HabitatSimActions.TURN_LEFT
            self._sim.step(sim_action)
            best_turn = (
                HabitatSimActions.TURN_LEFT
                if (
                    angle_between_quaternions(
                        grad_dir, self._sim.get_agent_state().rotation
                    )
                    < alpha
                )
                else HabitatSimActions.TURN_RIGHT
            )
            self._reset_agent_state(current_state)
            return self._get_return_value(best_turn)

    def _reset_agent_state(self, state: habitat_sim.AgentState) -> None:
        self._sim.set_agent_state(
            state.position, state.rotation, reset_sensors=False
        )

    def _geo_dist(self, goal_pos: np.array) -> float:
        return self._sim.geodesic_distance(
            self._sim.get_agent_state().position, [goal_pos]
        )

    def _est_max_grad_dir(self, goal_pos: np.array) -> np.array:

        current_state = self._sim.get_agent_state()
        current_pos = current_state.position

        if self.mode == "exact_gradient":
            points = self._sim.get_straight_shortest_path_points(
                self._sim.get_agent_state().position, goal_pos
            )
#             print("Curr Pos: {}".format(self._sim.get_agent_state().position))
#             print("Goal Pos: {}".format(goal_pos))
#             print("Length of path: {}".format(points))
#             print("length: {}".format(len(points)))
            # Add a little offset as things get weird if
            # points[1] - points[0] is anti-parallel with forward
            if len(points) < 2:
                return None
            
            max_grad_dir = quaternion_from_two_vectors(
                self._sim.forward_vector,
                points[1]
                - points[0]
                + EPSILON
                * np.cross(self._sim.up_vector, self._sim.forward_vector),
            )
            max_grad_dir.x = 0
            max_grad_dir = np.normalized(max_grad_dir)
        else:
            current_rotation = self._sim.get_agent_state().rotation
            current_dist = self._geo_dist(goal_pos)

            best_geodesic_delta = -2 * self._max_delta
            best_rotation = current_rotation
            for _ in range(0, 360, self._sim.config.TURN_ANGLE):
                sim_action = HabitatSimActions.MOVE_FORWARD
                self._sim.step(sim_action)
                new_delta = current_dist - self._geo_dist(goal_pos)

                if new_delta > best_geodesic_delta:
                    best_rotation = self._sim.get_agent_state().rotation
                    best_geodesic_delta = new_delta

                # If the best delta is within (1 - cos(TURN_ANGLE))% of the
                # best delta (the step size), then we almost certainly have
                # found the max grad dir and should just exit
                if np.isclose(
                    best_geodesic_delta,
                    self._max_delta,
                    rtol=1 - np.cos(np.deg2rad(self._sim.config.TURN_ANGLE)),
                ):
                    break

                self._sim.set_agent_state(
                    current_pos,
                    self._sim.get_agent_state().rotation,
                    reset_sensors=False,
                )

                sim_action = HabitatSimActions.TURN_LEFT
                self._sim.step(sim_action)

            self._reset_agent_state(current_state)

            max_grad_dir = best_rotation
            
        return max_grad_dir

    @property
    def mode(self):
        return self._mode

    @mode.setter
    def mode(self, new_mode: str):
        r"""Sets the mode for how the greedy follower determines the gradient of the geodesic_distance
        function (it then follows the negative of this gradient)

        :param new_mode: Must be one of 'exact_gradient' or 'approximate_gradient'.  If 'exact_gradient', then
                         the simulator must provide a straightend shortest path.

                         If 'approximate_gradient', the follower approximates the gradient by turning
                         and then stepping forward
        """
        assert new_mode in {"exact_gradient", "approximate_gradient"}
        if new_mode == "exact_gradient":
            assert (
                getattr(self._sim, "get_straight_shortest_path_points", None)
                is not None
            )
        self._mode = new_mode


In [9]:
def plot_image(obs):
    color_obs = obs["rgb"][:, :, :3]
    color_obs[190:195, 125:130, :] = [0, 0, 255] 
    depth_obs = obs["depth"]
    depth_obs = ((depth_obs - np.min(depth_obs))/ np.max(depth_obs) * 255.0).astype(np.int)
    depth_obs = np.stack([depth_obs]*3, axis=2)[:, :, :, 0]
    plt.imshow(np.concatenate([color_obs, depth_obs], axis=1))

In [10]:
def move_object(sim, object_id):
    sim.set_object_motion_type(MotionType.DYNAMIC, object_id)
    position = np.array(sim.get_translation(object_id))
    trans = sim.get_transformation(object_id)
    relative_trans = trans.transform_point(mn.Vector3([0, 1, 0]))
    sim.set_translation(relative_trans, object_id)
    sim.recompute_navmesh(sim.pathfinder, sim.navmesh_settings, True)
    new_position = np.array(sim.get_translation(object_id))
    
    return position, new_position

def reset_object(sim, object_id, position):
    sim.set_translation(position, object_id)
    sim.recompute_navmesh(sim.pathfinder, sim.navmesh_settings, True)
    sim.set_object_motion_type(MotionType.STATIC, object_id)

In [11]:
def shortest_path_example(config, env, mode):
    object_goal_radius = 1.0
    object_follower = ShortestPathFollower(env.habitat_env.sim, object_goal_radius, False)
    object_follower.mode = mode
    habitat_sim_obj = env.habitat_env.sim
    
    goal_radius = 0.75
    goal_follower = ShortestPathFollower(env.habitat_env.sim, goal_radius, False)
    goal_follower.mode = mode

    demonstrations = []
    for episode in range(len(env.habitat_env.episodes)):
        try:
            observations = env.reset()
            dem = {}
            dem['episode_id'] = env.habitat_env.current_episode.episode_id
            dem['actions'] = []
            dem['is_grab_success'] = True
            # print(dem['episode_id'])
            images = []
            
            position, new_position = move_object(habitat_sim_obj._sim, 1)
            
            while not env.habitat_env.episode_over:
                # position = np.array(env.habitat_env.sim._sim.get_translation(1))
                # position = env.habitat_env.current_episode.goals[0].position
                
                best_action = object_follower.get_next_action(
                    position
                )
                
                if best_action is None:                
                    break 

                dem['actions'].append(env.habitat_env.task.get_action_name(best_action))
                observations, reward, done, infos = env.step(action={'action': best_action})
                # print(observations)
                im = observations["rgb"]
                im[190:195, 125:130, :] = [0, 0, 255] 
                images.append(im)
            
            reset_object(habitat_sim_obj._sim, 1, position)
            observations = habitat_sim_obj.get_observations_at()
            im = observations["rgb"]
            im[190:195, 125:130, :] = [0, 0, 255] 
            images.append(im)

            # Call grab action
            observations, reward, done, infos = env.step(action={'action':4})
            im = observations["rgb"]
            im[190:195, 125:130, :] = [0, 0, 255] 
            images.append(im)

            dem['actions'].append(env.habitat_env.task.get_action_name(4))
            success = observations['gripped_object_id'] != -1

            if not success:
                dem['is_grab_success'] = False
                # images_to_video(images, 'data/videos', 'trajectory' + str(episode))
                # dem['videos'] = 'data/videos/trajectories' + str(episode)
                dem['infos'] = infos
                dem['reward'] = reward
                dem['success'] = False
                dem['done'] = done

                yield False, observations, reward, done, infos, dem, images

            while not env.habitat_env.episode_over:
                position = np.array(env.habitat_env.sim._sim.get_translation(1))
                best_action = goal_follower.get_next_action(
                    env.habitat_env.current_episode.goals[0].position
                )
                if best_action is None:
                    best_action = 4
                    dem['actions'].append(env.habitat_env.task.get_action_name(best_action))
                    observations, reward, done, infos = env.step(action={'action': best_action})
                    im = observations["rgb"]
                    im[190:195, 125:130, :] = [0, 0, 255] 
                    images.append(im)
                    break

                dem['actions'].append(env.habitat_env.task.get_action_name(best_action))
                observations, reward, done, infos = env.step(action={'action': best_action})
                im = observations["rgb"]
                im[190:195, 125:130, :] = [0, 0, 255] 
                images.append(im)

            # dem['videos'] = 'data/videos/trajectories' + str(episode)
            dem['infos'] = infos
            dem['reward'] = reward
            dem['success'] = True
            dem['done'] = done

            # images_to_video(images, 'data/videos', 'trajectory' + str(episode))
            yield True, observations, reward, done, infos, dem, images
        except Exception as e:
            print(str(e))
            yield False, observations, None, done, infos, dem, images
        

In [12]:
env.close()

NameError: name 'env' is not defined

In [12]:
env = SokobanRLEnv(config=config, dataset=dataset)

2020-06-30 18:47:44,729 initializing sim SokobanSim-v0


seed: 100


2020-06-30 18:47:46,784 Initializing task Sokoban-v0


Initializing Agents
Initializing Objects
Possible Actions: OrderedDict([('STOP', <habitat.tasks.nav.nav.StopAction object at 0x7fedcbdf90f0>), ('MOVE_FORWARD', <habitat.tasks.nav.nav.MoveForwardAction object at 0x7fedcbdf94e0>), ('TURN_LEFT', <habitat.tasks.nav.nav.TurnLeftAction object at 0x7fedcbdf91d0>), ('TURN_RIGHT', <habitat.tasks.nav.nav.TurnRightAction object at 0x7fedcbdf9748>), ('GRAB_RELEASE', <habitat.tasks.sokoban.sokoban_task.GrabOrReleaseAction object at 0x7fedcbdf9400>)])


In [ ]:
count = 0
demonstrations = []
episode_images = []
for i, (val, obs, reward, done, infos, dem, images) in enumerate(shortest_path_example(config.TASK_CONFIG, env, "approximate_gradient")):
    if val == False or done==False:
        count += 1
        print('----')
        print(dem['episode_id'])
        print('----')
        images_to_video(images, 'data/videos', 'trajectory'+str(dem['episode_id']))
    # print(obs)
    # print(obs['gripped_object_id'], infos, done, reward)
    print("i: {}, id:{}, success: {}, count={}, done={}, dist_to_goal: {}".format(
        i, dem['episode_id'], val, count, done, infos['object_distance_to_goal']
    ), end="\r")
    episode_images.append(images)
    demonstrations.append(dem)
    
    # plt.imshow(images[-1])
    
    # images_to_video(images, 'data/videos', 'trajectory'+str(dem['episode_id']))
    

----, id:1, success: True, count=0, done=True, dist_to_goal: {0: 0.4593968100827051}
2
----
----, id:2, success: True, count=1, done=False, dist_to_goal: {0: 0.274644216669889, 1: 7.0357391772669535}
3
----
----, id:3, success: True, count=2, done=False, dist_to_goal: {1: 2.179882833929407, 0: 2.1289546141440887}
4
----
----, id:4, success: True, count=3, done=False, dist_to_goal: {1: 4.1468295693248995, 0: 2.820923131170276}
5
----


In [138]:
# potential issues: 
# will not handle geodesic distances really well. #7743
# will not handle cases where the object is spawned too close. 

In [14]:
demonstrations_dict = {}
for demo in demonstrations:
    if 'infos' in demo:
        if 'object_goals' in demo['infos']:
            demo['infos']['object_goals'][1] = np.array(demo['infos']['object_goals'][1]).tolist()
        
    demonstrations_dict[demo['episode_id']] = demo

In [15]:
with gzip.open('data/datasets/sokoban/coda/v1/train/content/coda_hard.json.gz', "rt") as f:
    episodes = json.load(f)

In [16]:
'actions', 'is_grab_success', 'success', 'done'
for episode_id,demo  in demonstrations_dict.items():
    episode_id = demo['episode_id']
    episodes['episodes'][int(episode_id)]['demonstrations'] = demo

In [17]:
with gzip.open('data/datasets/sokoban/coda/v1/train/content/coda_hard_wdemo.json.gz', "wt") as f:
    json.dump(episodes, f)